In [1]:
import gym
import torch as th
import numpy as np
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate
import matplotlib.pyplot as plt


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
#wrapper for cont env with PPO, plot result for several steps
def ppo_eval_interval(p, L, t_t, n_iter, n_step, gae, learning_rate=0.0003):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1}
    PolicyCONFIG = dict(activation_fn=th.nn.Sigmoid,
                     net_arch=[dict(pi=[16, 16], vf=[64,64,64])])
    cont_env = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=4, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=0, gamma = 1, gae_lambda=gae,
                     learning_rate = learning_rate,use_sde = False, n_steps = n_step, policy_kwargs = PolicyCONFIG)
    env_eval = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    timesteps = 0
    numiter = n_iter#test
    res_mean_arr = []
    res_std_arr = []

    while(timesteps <= t_t):

        cont_model.learn(total_timesteps=2*4*n_step-1)#each iteration has 8192 timesteps with n_env=4
        timesteps = timesteps + 2*4*n_step

        res_mean, res_std = ppo_evaluate(cont_model, env_eval, numiter)
        res_mean_arr.append(-res_mean)
        res_std_arr.append(res_std)
        
        if -res_mean == min(res_mean_arr):
            cont_model.save("ppo_min_model_"+str(p)+"_"+str(L))
        
#         print(res_mean_arr)
    
    plt.plot(res_mean_arr)
    plt.xlabel("Iteration")
    plt.ylabel("Average cost")
    plt.title("L="+str(L)+", p="+str(p))
    plt.show()
    
    min_model = PPO.load("ppo_min_model_"+str(p)+"_"+str(L))
    mean_min, std_min = ppo_evaluate(min_model, env_eval, 50000)
    print("p="+str(p)+"， L="+str(L)+": mean "+str(-mean_min)+", std_dev: "+str(std_min))
    
    return res_mean_arr, res_std_arr, mean_min, std_min

In [4]:
# listp = [0.25,1,4,9,39,99]
# listL = [4,70]
listp = [0.25,1,4,9,39,99]
listL = [1]
# t_t = 100000
n_iter = 1000
gae_lambda = 0.95
# n_step = 8192
learning_rate = 0.0003
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        n_step = 64*L 
        t_t = 300*4*n_step
        res_mean, res_std, mean_min, std_min = ppo_eval_interval(p,L,t_t, n_iter, n_step, gae_lambda, learning_rate)
        ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':-mean_min, 'res_std': std_min}, ignore_index=True)

Running PPO w/: p= 0.25 , L= 1
mean:  -0.41301281369892406
standard deviation: 0.025152140082046883
mean:  -0.3688236956460198
standard deviation: 0.015932811226133877
mean:  -0.3703892796821026
standard deviation: 0.020804202556020283
mean:  -0.3034406879623701
standard deviation: 0.009991109493865015
mean:  -0.2894439643853438
standard deviation: 0.007610894547374724
mean:  -0.2762250314301494
standard deviation: 0.010552848307674681
mean:  -0.2696801303162928
standard deviation: 0.0019456390178024764
mean:  -0.2758508940064452
standard deviation: 0.01251274402274999
mean:  -0.26026567878910134
standard deviation: 0.009981266418160783
mean:  -0.26611127123672923
standard deviation: 0.0047934744371867875
mean:  -0.2639837321589017
standard deviation: 0.011276576939559354
mean:  -0.25411959314105526
standard deviation: 0.006786425265161383
mean:  -0.25043423275640525
standard deviation: 0.007062345550363097
mean:  -0.2586099036104439
standard deviation: 0.00785343858779233
mean:  -0.25

KeyboardInterrupt: 

In [ ]:
ppo_res

In [ ]:
plt.plot(res_mean)

## Below is obsolete.